In [1]:
# 加载向量库和检索链
from langchain.vectorstores import Chroma
from langchain_community.embeddings import QianfanEmbeddingsEndpoint # 调用 百度文心一言 的 Embeddings 模型
from dotenv import load_dotenv, find_dotenv
import os
# _ = load_dotenv(find_dotenv()) # read local .env file
_ = load_dotenv('../QA_Project/.env')

# 获取环境变量 
wenxin_api_key = os.environ["wenxin_api_key"]
wenxin_secret_key = os.environ["wenxin_secret_key"]


# 定义 Embeddings
qianfan_embedding = QianfanEmbeddingsEndpoint(qianfan_ak=wenxin_api_key,
                                  qianfan_sk=wenxin_secret_key) 

# 向量数据库持久化路径
persist_directory = '../QA_Project/data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=qianfan_embedding
)


In [11]:
from langchain.prompts import PromptTemplate

template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)

from langchain.chains import RetrievalQA
from langchain_community.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint
import logging

# 获取特定库的日志器，设置日志级别为ERROR，这样低于ERROR级别的日志（如WARNING, INFO, DEBUG）将不会被打印
logger_qfllm = logging.getLogger('QianfanChatEndpoint')
logger_qfllm.setLevel(logging.ERROR)
logger_qf = logging.getLogger('qianfan')
logger_qf.setLevel(logging.ERROR)

llm = QianfanChatEndpoint(
    qianfan_ak=wenxin_api_key,
    qianfan_sk=wenxin_secret_key,model='ERNIE-Bot-4'
)

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


question = "南瓜书和西瓜书有什么关系？"
result = qa_chain({"query": question})
print("问题一：")
print(result["result"])


question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
print("问题二：")
print(result["result"])

问题一：
南瓜书是西瓜书的补充和辅助教材，它以西瓜书的内容为前置知识进行表述，旨在帮助读者更好地理解和掌握西瓜书中的机器学习算法和公式。两本书可以结合使用，以西瓜书为主线，遇到难题时再查阅南瓜书进行辅助理解。谢谢你的提问！
问题二：
南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书。对于初学机器学习的小白，建议先从西瓜书入手，简单过一下第1章和第2章的公式，等有一定基础后再深入学习南瓜书中的详细推导和解释。谢谢你的提问！


In [12]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v2)

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


question = "南瓜书和西瓜书有什么关系？"
result = qa_chain({"query": question})
print("问题一：")
print(result["result"])

question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
print("问题二：")
print(result["result"])

问题一：
南瓜书和西瓜书都是与机器学习相关的教材或参考资料。从给定的上下文中，我们可以推断出以下几点关系：

1. 内容关联：南瓜书的内容是以西瓜书的内容为前置知识进行表述的。这意味着，南瓜书可能是对西瓜书中某些内容、公式或理论的进一步解释、展开或补充。因此，当读者在阅读西瓜书时遇到难以理解或推导的部分，可以查阅南瓜书以获取更详细的说明。
2. 使用建议：建议读者以西瓜书为主线进行学习，在遇到难题时再参考南瓜书。这表明，西瓜书可能是一个更全面、系统的机器学习教材，而南瓜书则更像是一个辅助工具或参考手册。
3. 命名风格：从上下文中可以看出，南瓜书和西瓜书在命名上有一定的相似性，可能寓意着它们之间的某种关联或连续性。实际上，这种命名风格可能是为了吸引读者的注意，使教材更易于记忆和识别。

综上所述，南瓜书和西瓜书在内容和使用上具有一定的互补性，共同构成了机器学习领域的一套参考资料。读者可以根据自己的学习需求和兴趣，结合使用这两本书进行学习。
问题二：
根据给出的上下文，南瓜书《机器学习公式详解》的使用方法主要如下：

1. 以西瓜书为主线进行学习。南瓜书的内容是基于西瓜书的内容进行表述的，因此，最好先以西瓜书为主线进行学习。在阅读西瓜书的过程中，如果遇到自己推导不出来或者看不懂的公式，可以查阅南瓜书进行辅助理解。
2. 不要过早深究某些公式。对于初学机器学习的小白，西瓜书第1章和第2章的公式不建议一开始就深究，可以先简单过一下，等学到一定程度后再回来深入理解。同样，南瓜书中的一些公式和推导也可以按照这种策略进行学习。
3. 注意决策树等直观算法的理解。虽然南瓜书主要关注于公式的详解，但也要注意理解算法本身的直观含义。例如，决策树算法背后没有复杂的数学推导，更符合人类日常思维方式，理解起来也更为直观。在学习这类算法时，可以结合书中的例子进行理解。
4. 作为笔记和参考。南瓜书本质上是作者自学时记下来的笔记，因此也可以作为学习者自己的笔记和参考。在学习过程中，可以在南瓜书上做笔记、标记重点、写下自己的理解等，以帮助自己更好地学习和掌握机器学习的知识。

总的来说，南瓜书《机器学习公式详解》是一本辅助学习的书籍，主要以西瓜书为主线进行学习，在遇到困难和疑惑时查阅南瓜书进行辅助理解和参考。同时也要注意理解算法本身的直观含义，以及做好自己的笔记和参考。


In [13]:
print("问题：")
question = "应该如何使用南瓜书？"
print(question)
print("模型回答：")
result = qa_chain({"query": question})
print(result["result"])

问题：
应该如何使用南瓜书？
模型回答：
根据提供的上下文，南瓜书《机器学习公式详解》的使用方法如下：

1. 南瓜书的内容是以西瓜书的内容为前置知识进行表述的，因此最佳使用方法是以西瓜书为主线进行学习。当遇到自己推导不出来或者看不懂的公式时，再来查阅南瓜书进行辅助理解。
2. 对于初学机器学习的小白，西瓜书第1章和第2章的公式不建议深究，可以简单过一遍，等学到一定程度后再回来仔细研究。这一过程中，如果遇到难以理解的公式或推导，也可以参考南瓜书的相关内容。
3. 南瓜书的目标是帮助读者成为一名合格的“理工科数学基础扎实点的大二下学生”，因此在使用南瓜书时，应着重关注书中的推导细节和具体解释，以加深对机器学习算法和数学原理的理解。

总的来说，南瓜书《机器学习公式详解》是一本辅助性的学习资料，适合与西瓜书配合使用，以帮助读者更好地理解和掌握机器学习的相关知识和算法。


In [18]:
print(result["source_documents"])


[Document(page_content='等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第 1 章和第 2 章的公式强烈不建议深究，简单过一下即可，等你学得\n有点飘的时候再回来啃都来得及；', metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../QA_Project/data_base/knowledge_db/pumpkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}), Document(page_content='具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明', metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaT

In [19]:
prompt_template = '''
请你做如下选择题：
题目：南瓜书的作者是谁？
选项：A 周志明 B 谢文睿 C 秦州 D 贾彬彬
你可以参考的知识片段：
```
{}
```
请仅返回选择的选项
如果你无法做出选择，请返回空
'''


In [20]:
def multi_select_score_v1(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return 0
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    else:
        # 漏选
        return 0.5


In [21]:
answer1 = 'B C'
answer2 = '西瓜书的作者是 A 周志华'
answer3 = '应该选择 B C D'
answer4 = '我不知道'
true_answer = 'BCD'
print("答案一得分：", multi_select_score_v1(true_answer, answer1))
print("答案二得分：", multi_select_score_v1(true_answer, answer2))
print("答案三得分：", multi_select_score_v1(true_answer, answer3))
print("答案四得分：", multi_select_score_v1(true_answer, answer4))

答案一得分： 0.5
答案二得分： 0
答案三得分： 1
答案四得分： 0


In [22]:
def multi_select_score_v2(true_answer : str, generate_answer : str) -> float:
    # true_anser : 正确答案，str 类型，例如 'BCD'
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(true_answer)
    '''为便于计算，我们假设每道题都只有 A B C D 四个选项'''
    # 先找出错误答案集合
    false_answers = [item for item in ['A', 'B', 'C', 'D'] if item not in true_answers]
    # 如果生成答案出现了错误答案
    for one_answer in false_answers:
        if one_answer in generate_answer:
            return -1
    # 再判断是否全选了正确答案
    if_correct = 0
    for one_answer in true_answers:
        if one_answer in generate_answer:
            if_correct += 1
            continue
    if if_correct == 0:
        # 不选
        return 0
    elif if_correct == len(true_answers):
        # 全选
        return 1
    else:
        # 漏选
        return 0.5

In [23]:
answer1 = 'B C'
answer2 = '西瓜书的作者是 A 周志华'
answer3 = '应该选择 B C D'
answer4 = '我不知道'
true_answer = 'BCD'
print("答案一得分：", multi_select_score_v2(true_answer, answer1))
print("答案二得分：", multi_select_score_v2(true_answer, answer2))
print("答案三得分：", multi_select_score_v2(true_answer, answer3))
print("答案四得分：", multi_select_score_v2(true_answer, answer4))

答案一得分： 0.5
答案二得分： -1
答案三得分： 1
答案四得分： 0


In [25]:
from nltk.translate.bleu_score import sentence_bleu
import jieba

def bleu_score(true_answer : str, generate_answer : str) -> float:
    # true_anser : 标准答案，str 类型
    # generate_answer : 模型生成答案，str 类型
    true_answers = list(jieba.cut(true_answer))
    # print(true_answers)
    generate_answers = list(jieba.cut(generate_answer))
    # print(generate_answers)
    bleu_score = sentence_bleu(true_answers, generate_answers)
    return bleu_score

In [28]:
true_answer = '周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。'

print("答案一：")
answer1 = '周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。'
print(answer1)
score = bleu_score(true_answer, answer1)
print("得分：", score)
print("答案二：")
answer2 = '本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二下学生”'
print(answer2)
score = bleu_score(true_answer, answer2)
print("得分：", score)

答案一：
周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。
得分： 1.2705543769116016e-231
答案二：
本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二下学生”
得分： 1.1935398790363042e-231


In [29]:
prompt = '''
你是一个模型回答评估员。
接下来，我将给你一个问题、对应的知识片段以及模型根据知识片段对问题的回答。
请你依次评估以下维度模型回答的表现，分别给出打分：

① 知识查找正确性。评估系统给定的知识片段是否能够对问题做出回答。如果知识片段不能做出回答，打分为0；如果知识片段可以做出回答，打分为1。

② 回答一致性。评估系统的回答是否针对用户问题展开，是否有偏题、错误理解题意的情况，打分分值在0~1之间，0为完全偏题，1为完全切题。

③ 回答幻觉比例。该维度需要综合系统回答与查找到的知识片段，评估系统的回答是否出现幻觉，打分分值在0~1之间,0为全部是模型幻觉，1为没有任何幻觉。

④ 回答正确性。该维度评估系统回答是否正确，是否充分解答了用户问题，打分分值在0~1之间，0为完全不正确，1为完全正确。

⑤ 逻辑性。该维度评估系统回答是否逻辑连贯，是否出现前后冲突、逻辑混乱的情况。打分分值在0~1之间，0为逻辑完全混乱，1为完全没有逻辑问题。

⑥ 通顺性。该维度评估系统回答是否通顺、合乎语法。打分分值在0~1之间，0为语句完全不通顺，1为语句完全通顺没有任何语法问题。

⑦ 智能性。该维度评估系统回答是否拟人化、智能化，是否能充分让用户混淆人工回答与智能回答。打分分值在0~1之间，0为非常明显的模型回答，1为与人工回答高度一致。

你应该是比较严苛的评估员，很少给出满分的高评估。
用户问题：
```
{}
```
待评估的回答：
```
{}
```
给定的知识片段：
```
{}
```
你应该返回给我一个可直接解析的 Python 字典，字典的键是如上维度，值是每一个维度对应的评估打分。
不要输出任何其他内容。
'''


In [30]:
import requests
import json

def get_access_token():
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """
    # 指定网址
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=NSwW3Zr7f1rW8rRByKyqPKOV&client_secret=rbt3kGXuophuE8XOm45A5M53qft8p6c9"
    # 设置 POST 访问
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    # 通过 POST 访问获取账户对应的 access_token
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")
# 一个封装 Wenxin 接口的函数，参数为 Prompt，返回对应结果
def get_completion_weixin(prompt, temperature = 0.95, access_token = ""):
    '''
    prompt: 对应的提示词
    temperature：温度系数
    access_token：已获取到的秘钥
    '''
    url = f"https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions_pro?access_token={access_token}"
    # 配置 POST 参数
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ],
        "temperature" : temperature
    })
    headers = {
        'Content-Type': 'application/json'
    }
    # 发起请求
    response = requests.request("POST", url, headers=headers, data=payload)
    # 返回的是一个 Json 字符串
    js = json.loads(response.text)
    # print(js)
    return js["result"]


In [32]:
question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
answer = result["result"]
knowledge = result["source_documents"]

response = get_completion_weixin(prompt.format(question, answer, knowledge),access_token=get_access_token())
print(response)


```python
{
    "知识查找正确性": 1,
    "回答一致性": 1,
    "回答幻觉比例": 1,
    "回答正确性": 1,
    "逻辑性": 1,
    "通顺性": 1,
    "智能性": 0.9
}
```
